In [2]:
import numpy as np # 今回は明示的には使わない
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb # LightGBM でモデルを作成する
from sklearn.metrics import accuracy_score, roc_auc_score #精度評価に使用

In [11]:
# 学習データの読み込み
train_df =  pd.read_csv("../input/tdse-inclass-competition-0/train.csv")
# 特徴量。Surname, Geography, Gender はそのままでは使えないので削除
X_train_df = train_df.drop(
    ["Exited", "Surname", "Geography", "Gender"],
    axis=1,
) 
# 目的変数
y_train_df = train_df["Exited"]

# テストデータの読み込み
test_df =  pd.read_csv("../input/tdse-inclass-competition-0/test.csv")
# 同様にSurname, Geography, Gender を削除
X_test_df = test_df.drop(
    ["Surname", "Geography", "Gender"],
    axis=1,
)

# LightGBM用のデータセット作成
train_data = lgb.Dataset(X_train_df, label=y_train_df)

In [9]:
# モデルのパラメータ設定
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# モデルのトレーニング
model = lgb.train(
    params,
    train_data,
)

In [22]:
# テストデータに対する予測
y_pred_np = model.predict(X_test_df)

In [24]:
y_pred_np

array([0.01599476, 0.78082902, 0.02689072, ..., 0.02765597, 0.11213857,
       0.22836047])

In [27]:
# 提出用ファイルの作成
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "Exited": y_pred_np,
})

In [28]:
submission_df.head()

,id,Exited
0,165034,0.015995
1,165035,0.780829
2,165036,0.026891
3,165037,0.302446
4,165038,0.259043


In [ ]:
submission_df.to_csv("submission.csv", index=False)